#### libics: drivers

# Stanford Research Systems SR760 FFT Analyzer

Product homepage: [SRS FFT Analyzer](https://www.thinksrs.com/products/sr760770.html)

### Imports

In [ ]:
import os

from libics.drv import drv
from libics.drv.itf import itf
from libics.data import types, dataset, stp
from libics.display import plot, plotdefault
from libics.file import hdf

### Configuration

In [ ]:
itf_cfg = {
    "protocol": itf.ITF_PROTOCOL.TEXT,
    "interface": itf.ITF_TXT.ETHERNET,
    "address": "130.183.96.12",
    "buffer_size": 4096,
    "send_timeout": 1.0,
    "send_termchar": "\r\n",
    "recv_timeout": 7.0,
    "recv_termchar": "\n",
    "txt_ethernet_type": itf.TXT_ETHERNET_TYPE.GPIB,
    "port": 1234,
    "blocking": True,
    "gpib_mode": itf.TXT_ETHERNET_GPIB.MODE.CONTROLLER,
    "gpib_address": 5,
    "ctrl_model": itf.TXT_ETHERNET_GPIB.MODEL.PROLOGIX_GPIB_ETHERNET
}
itf_cfg = itf.ProtocolCfgBase(**itf_cfg).get_hl_cfg()

In [ ]:
drv_cfg = {
    "driver": drv.DRV_DRIVER.SPAN,
    "interface": itf_cfg,
    "identifier": "test_stanford_sr760",
    "model": drv.DRV_MODEL.STANFORD_SR760,
    "bandwidth": 250.0,
    "frequency_start": 0.0,
    "frequency_stop": 100e3,
    "average_mode": drv.DRV_SPAN.AVERAGE_MODE.LIN,
    "average_count": 2000
}
drv_cfg = drv.DrvCfgBase(**drv_cfg).get_hl_cfg()

In [ ]:
photodiode_model = types.ValQuantity(
    name="photodiode model", val="Thorlabs PDA100A-EC"
)
photodiode_gain = types.ValQuantity(
    name="photodiode gain", unit="dB", val=0
)
photodiode_voltage_dc = types.ValQuantity(
    name="photodiode voltage dc", symbol="V_dc", unit="V", val=1.0
)
light_power = types.ValQuantity(
    name="light power", symbol="P", unit="mW", val=10.0
)
stp_cfg = stp.SetupCfgBase(
    photodiode_model=photodiode_model, photodiode_gain=photodiode_gain,
    photodiode_voltage_dc=photodiode_voltage_dc, light_power=light_power
)

In [ ]:
# File path of saved PSD data set (without file extension)
file_path = os.path.join(
    os.environ["USERPROFILE"], "Desktop",
    "sample_psd"
)

### Setup

In [ ]:
span = drv.DrvBase(cfg=drv_cfg).get_drv()
psd = None

### Script

##### Acquire data

In [ ]:
try:
    with span:
        print("Read power spectral density")
        psd = span.read_powerspectraldensity(read_meta=True)
except Exception as e:
    print("ERROR: check Ethernet/GPIB connection")
    print(e)

##### Plot data

In [ ]:
fig_cfg = plotdefault.get_figurecfg()
plot_cfg = plotdefault.get_plotcfg_arraydata_1d()
fig = plot.Figure(fig_cfg, plot_cfg, data=psd)
fig.plot()

##### Save data

In [ ]:
data_set = dataset.DataSet(data=psd, cfg=drv_cfg, stp=stp_cfg)
hdf.write_hdf(data_set, file_path=(file_path + ".hdf5"))
hdf.write_json(data_set, file_path + ".json")